# Filings Monthly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import psycopg2
import pandas as pd
import matplotlib
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from IPython.core.display import HTML

%load_ext sql
%config SqlMagic.displaylimit = 5
%config SqlMagic.style = '_DEPRECATED_DEFAULT'


This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
connect_to_db = 'postgresql://' + \
                os.getenv('PG_USER', '') + ":" + os.getenv('PG_PASSWORD', '') +'@' + \
                os.getenv('PG_HOST', '') + ':' + os.getenv('PG_PORT', '5432') + '/' + os.getenv('PG_DB_NAME', '');
    
%sql $connect_to_db

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

monthly total before running time.

In [ ]:
%%sql stat_filings_monthly_completed  <<
WITH Detail AS 
(
    SELECT b.identifier AS COOPERATIVE_NUMBER
           , b.legal_name AS COOPERATIVE_NAME
           , COUNT(b.identifier) AS FILINGS_TOTAL_COMPLETED
           , STRING_AGG(f.filing_type, ', ')  AS FILING_TYPES_COMPLETED
    FROM businesses b,
    filings f
    WHERE b.id = f.business_id
    AND b.legal_type='CP'
    AND f.status='COMPLETED'
    AND date(f.completion_date at time zone 'utc' at time zone 'pst')  > date(current_date - 1 - interval '1 months')    
    AND date(f.completion_date at time zone 'utc' at time zone 'pst')  <= date(current_date - 1)
    GROUP BY b.identifier, b.legal_name    
)

SELECT * FROM Detail
UNION ALL
SELECT 'SUM' identifier, null, sum(FILINGS_TOTAL_COMPLETED) AS count, null
from Detail;


In [ ]:
edt = stat_filings_monthly_completed.DataFrame()

Save to CSV

In [ ]:
last_month = datetime.now() - relativedelta(months=1)
filename = os.path.join(os.getcwd(), r'data/')+'coop_filings_monthly_stats_for_' + format(last_month, '%B_%Y') +'.csv'
edt.to_csv(filename, sep=',', encoding='utf-8', index=False)